In [32]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [33]:
print("download.default_directory=" + os.getcwd() + "\Downloads")

download.default_directory=C:\Users\marti\Desktop\Webscraping\Downloads


In [34]:
options = webdriver.ChromeOptions() 
prefs = {
    "download.default_directory": os.getcwd() + "\Downloads",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True
}

options.add_experimental_option('prefs', prefs)

In [35]:
start_page = "https://www.researchgate.net/search/authors?q"
test_name = "Michael Kabo"

https://www.researchgate.net/search/authors?q


In [36]:
#Use pandas create dataframe (df) object that will store the data
columns = ['name','paper_type','citation','url','abstract','title','collaborators']
citations = pd.DataFrame()
#Place the list of names generated previously in the project into the df object
citations

""


In [37]:
#df1['e'] = Series(np.random.randn(sLength), index=df1.index)
#citations['a'] = pd.Series(test_name, index=citations.index)


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [48]:
#def research_gate_scraper(url):
    
research_gate = "https://www.researchgate.net/search/authors?q"

#Instatiate the webdriver and set to implicitly wait for page objects to load
driver = webdriver.Chrome()
driver.implicitly_wait(10) # seconds loading implicitly

#Set driver to go to start_page and search a given name
driver.get(research_gate)
print(driver.current_url)
search_bar = driver.find_element_by_tag_name('input')
search_bar.send_keys(test_name)
search_bar.send_keys(Keys.ENTER)


#Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
view_profile = driver.find_element_by_link_text('View Profile')
view_profile.click()

#Attempt to find the total number of research items for a given professsor 
num_research_items = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit"]')
num_research_items = num_research_items.text
num_research_items = int(num_research_items)
print(num_research_items)


# grab the research interests
interests = driver.find_elements_by_xpath('//*[@id="about"]/div/div/div[3]/div/div/div[2]/div/div[2]/div/div/div/div/div')
interests = [item.text for item in interests]
while '' in interests:
    interests.remove('')
print(interests)

# Retrieve all the urls of the first page
research_a_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
urls = [tag.get_attribute("href") for tag in research_a_tags]
titles = [tag.text for tag in research_a_tags]
paper_types = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
paper_types = [item.text for item in paper_types]
num_research_items -= 100

current_url = driver.current_url

page_count = 2
while num_research_items > 0:
    driver.get(current_url + '/' + str(page_count))
    temp_tags = driver.find_elements_by_xpath('//div[@id="research"]//div/a[@class="nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare"]')
    for tag in temp_tags:
        urls.append(tag.get_attribute("href"))
        titles.append(tag.text)
    temp_papers = driver.find_elements_by_xpath('//div[@id="research"]//a[@class="nova-e-badge nova-e-badge--color-green nova-e-badge--display-block nova-e-badge--luminosity-high nova-e-badge--size-l nova-e-badge--theme-solid nova-e-badge--radius-m nova-v-publication-item__type"]')
    for paper in temp_papers:
        paper_types.append(paper.text)
    num_research_items -= 100
    page_count += 1

while "Full-text available" in paper_types:
    paper_types.remove("Full-text available")
    
driver.close()
    

https://www.researchgate.net/search/authors?q
112
['Mechanical Testing', 'Numerical Analysis', 'Biomechanics', 'Mechanical Engineering', 'Fracture', 'Applied Mechanics', 'Arthroplasty', 'Microsoft Office', 'Orthopedic Biomechanics', 'Hip And Knee Arthroplasty']


In [39]:
for url in urls:
    print(url)
    
print(len(urls))

https://www.researchgate.net/publication/315416439_Composite_mechanics_of_the_multilayer_structure_of_the_annulus_fibrosus
https://www.researchgate.net/publication/313626183_Modeling_and_Analysis_of_Heart_Left_Ventricle_in_Diastole_Using_Finite_Element
https://www.researchgate.net/publication/285611966_Machine_design_Redesigned
https://www.researchgate.net/publication/261035675_Endoprosthetic_reconstruction_after_resection_of_musculoskeletal
https://www.researchgate.net/publication/40768832_Cemented_Distal_Femoral_Endoprostheses_for_Musculoskeletal_Tumor_Improved_Survival_of_Modular_versus_Custom_Implants
https://www.researchgate.net/publication/44571522_How_Long_Do_Endoprosthetic_Reconstructions_for_Proximal_Femoral_Tumors_Last
https://www.researchgate.net/publication/44624077_Cemented_Endoprosthetic_Reconstruction_of_the_Proximal_Tibia_How_Long_Do_They_Last
https://www.researchgate.net/publication/51381639_Hysteretic_pinching_of_human_secondary_osteons_subjected_to_torsion
https://ww

In [44]:
count = 0
#for url in urls:
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10) # seconds loading implicitly

#time.sleep(randint(5,10))

driver.get(urls[0])

# At this point we already have name, title, type, url
# Need citation, collabs, date, DOI, publisher, edition, series, volume, issue, ISSN, ISBN, handle, book title, 



# every print is something to be added to the dataframe object

if paper_types[0] == 'Article':
    
    # Grab the DOI
    meta = driver.find_element_by_xpath('//*[@id="lite-page"]/main/section/section[1]/div[1]/div[2]')
    doi = meta.text.split(" ")
    doi = doi[1]
    
    # get the abstract seperately
    abstract = driver.find_element_by_xpath('//div[@class="nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-auto nova-e-text--color-inherit"]')    
    abstract = abstract.text
    print(abstract)
    
    # then we'll get the collaborators
    collab = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/ul/li/div/div/div/div/div[2]/div/div/div/div/div/a')
    collab = [person.text for person in collab]
    print(collab)
    
    # Attempt to get the date
    date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
    date = date.text
    print(date)
    
    #download the citatation
    download_link = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/a')
    download_link.click()
    
    # click the text button
    text_radio_btn = driver.find_element_by_xpath("//span[contains(.,' Plain Text')]")
    text_radio_btn.click()
    
    # download the citation file
    download = driver.find_element_by_xpath('//button[@class="nova-c-button nova-c-button--align-center nova-c-button--radius-m nova-c-button--size-m nova-c-button--color-blue nova-c-button--theme-solid nova-c-button--width-auto"]')
    download.click()
    
    time.sleep(3)
    
    driver.close()
    
    
    # now that file is downloaded, need to open the file
    
    # here I grab the file name; the trick to this is making sure that its the only file in downloads
    citation_file = os.listdir(os.getcwd() + "\Downloads")[0]
        
    # grab the citation
    f = open("Downloads\\"+citation_file, "r")
    citation = f.read()
    print(citation)
    f.close()
    
    # delete it
    os.remove("Downloads\\"+citation_file)
    
    # build a pandas row
    
    # add it to the main dataframe
    
    # end loop

    
#elif paper_type[0] == "Conference Paper"





#driver.close()
    #count +=1

The degeneration of the intervertebral disc (IVD) rests on many factors, but some of the main ones are the absence of a blood vessel supply, excessive overloading, and cycling loadings. This deterioration may contribute to peripheral pain and neurological dysfunctions. When degeneration is severe or in some other cases when the annulus fibrosus (AF) ruptures, fusion surgery, disc arthroplasty, and disc discectomy have been used as clinical solutions. However, these solutions limit the range of motion, the material used does not reproduce the natural behavior, and more importantly, it weakens the overall mechanical response of the spinal column, which accelerates degeneration. Recent research indicates a new focus on the composite nature of the annulus. In this paper, a mathematical model is used to predict the mechanical behavior of the AF based on its anatomical structure. Previously published experimental data of AF are correlated with the mechanistic-based predictions and found to b

In [99]:
counter = 0
for url in urls:
    print(counter, ":", url)
    counter +=1

0 : https://www.researchgate.net/publication/315416439_Composite_mechanics_of_the_multilayer_structure_of_the_annulus_fibrosus
1 : https://www.researchgate.net/publication/313626183_Modeling_and_Analysis_of_Heart_Left_Ventricle_in_Diastole_Using_Finite_Element
2 : https://www.researchgate.net/publication/285611966_Machine_design_Redesigned
3 : https://www.researchgate.net/publication/261035675_Endoprosthetic_reconstruction_after_resection_of_musculoskeletal
4 : https://www.researchgate.net/publication/40768832_Cemented_Distal_Femoral_Endoprostheses_for_Musculoskeletal_Tumor_Improved_Survival_of_Modular_versus_Custom_Implants
5 : https://www.researchgate.net/publication/44571522_How_Long_Do_Endoprosthetic_Reconstructions_for_Proximal_Femoral_Tumors_Last
6 : https://www.researchgate.net/publication/44624077_Cemented_Endoprosthetic_Reconstruction_of_the_Proximal_Tibia_How_Long_Do_They_Last
7 : https://www.researchgate.net/publication/51381639_Hysteretic_pinching_of_human_secondary_osteons


Note that ResearchGate only loads 100 citations per page, if number of citations % 100 = 0 we go to second page

In [8]:
counter = 1
for title in titles:
    print(counter,":",title)
    counter += 1

1 : Composite mechanics of the multilayer structure of the annulus fibrosus
2 : Modeling and Analysis of Heart Left Ventricle in Diastole Using Finite Element
3 : Machine design: Redesigned
4 : Endoprosthetic reconstruction after resection of musculoskeletal.
5 : Cemented Distal Femoral Endoprostheses for Musculoskeletal Tumor: Improved Survival of Modular versus Custom Implants
6 : How Long Do Endoprosthetic Reconstructions for Proximal Femoral Tumors Last?
7 : Cemented Endoprosthetic Reconstruction of the Proximal Tibia: How Long Do They Last?
8 : Hysteretic pinching of human secondary osteons subjected to torsion
9 : Response to Letter to the Editor by Shyh-Jen Wang
10 : Mathematical Modeling of Human Secondary Osteons
11 : Proximal Interphalangeal Joint Stiffness: Measurement and Analysis
12 : The mechanical and thermal effects of focused ultrasound in a model biological material
13 : Healing of Critically Sized Femoral Defects, Using Genetically Modified Mesenchymal Stem Cells fro

https://csun.edu/engineering-computer-science/research-faculty-resources

In [ ]:
citation = {
    name: "",
    citation_text: "",
    title: "",
    type: "",
    abstract: "",
    book_title: "",
    journal_title: "",
    date : "",
    collaborators: "",
    publisher: "",
    edition: "",
    series: "",
    volume: "",
    issue: "",
    chapter: "",
    pages: "",
    doi: "",
    issn: "",
    isbn : "",
    handle: "",
    url : ""
}

profile = {
    name : "",
    bio : "",
    dept : "",
    personal : "",
    academic : "",
    research : "",
    orcid : "",
    scopus : ""
}